In [1]:
from gwpy.frequencyseries import FrequencySeries
from gwpy.timeseries import TimeSeries
import astropy.units as u
import numpy as np

/home/user/miniconda3/envs/gw_signal_tools/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(True)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


In [2]:
test = FrequencySeries(
    [1,2,3],
    frequencies=[-1,0,1] * u.dimensionless_unscaled
)

test.frequencies /= u.Hz

test.frequencies

<Index [-1.,  0.,  1.] 1 / Hz>

In [3]:
print(test.interpolate(0.01))
print((test * u.kg).interpolate(0.01))

FrequencySeries([1.  , 1.01, 1.02, ..., 2.98, 2.99, 3.  ]
                unit: dimensionless,
                f0: -1.0 1 / Hz,
                df: 0.01 1 / Hz,
                epoch: None,
                name: None,
                channel: None)
FrequencySeries([1.  , 1.01, 1.02, ..., 2.98, 2.99, 3.  ]
                unit: kg,
                f0: -1.0 1 / Hz,
                df: 0.01 1 / Hz,
                epoch: None,
                name: None,
                channel: None)


Hmmm, interpolate does not retain units...

-> fixed it

In [4]:
print(test.crop(end=0))

FrequencySeries([1]
                unit: dimensionless,
                f0: -1.0 1 / Hz,
                df: 1.0 1 / Hz,
                epoch: None,
                name: None,
                channel: None)


In [5]:
print(isinstance(1.0, u.Quantity))
print(isinstance(1.0*u.dimensionless_unscaled, u.Quantity))

False
True


In [6]:
print(isinstance(test, FrequencySeries))
print(isinstance(test, TimeSeries))

True
False


## Testing gw_signal_tools Functoins

In [7]:
from gw_signal_tools.inner_product import inner_product

In [8]:
inner_product(test, test, test * 1/u.Hz)

2024-01-31  15:20:12  INFO (inner_product.py: 303): , , , 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 410): , , , 1 / Hz


15.2578125

In [9]:
from scipy.integrate import simpson

4.0 * np.real(simpson(y=test * test.conjugate() / test, x=test.frequencies))

16.0

In [10]:
inner_product(test, test, test / u.Hz, df=test.df)

2024-01-31  15:20:12  INFO (inner_product.py: 303): , , , 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 410): , , , 1 / Hz


6.0

In [11]:
inner_product(test, test, test / u.Hz, df=0.001 / u.Hz)

2024-01-31  15:20:12  INFO (inner_product.py: 303): , , , 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 410): , , , 1 / Hz


15.988002000000016

In [12]:
inner_product(test, test, test / u.Hz, df=0.001 / u.Hz, f_range=[0, None])

2024-01-31  15:20:12  INFO (inner_product.py: 303): , , , 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 410): , , , 1 / Hz


9.988002

In [13]:
# inner_product(test, test, test, df=0.001 / u.Hz, optimize_time_and_phase=True)
# Gives error, but this is intended

In [14]:
inner_product(test, test, test / u.Hz, df=0.001 / u.Hz, f_range=[0, None], optimize_time_and_phase=True)

2024-01-31  15:20:12  INFO (inner_product.py: 303): , , , 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 410): , , , 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 425): , , , 1 / Hz


(<TimeSeries([0.003999-0.00600215j, 0.003999+0.00598959j,
              0.003999-0.00601476j, ..., 0.003999+0.00601476j,
              0.003999-0.00598959j, 0.003999+0.00600215j]
             unit=Unit(dimensionless),
             t0=<Quantity -499.74987494 Hz>,
             dt=<Quantity 0.50025013 Hz>,
             name=None,
             channel=None)>,
 9.998000000000001,
 <Quantity -2.38742359e-11 Hz>)

In [15]:
test_f_series = FrequencySeries(
    np.arange(1, step=0.1),
    frequencies=np.arange(1, step=0.1)
)

test_t_series = TimeSeries(
    np.arange(1, step=0.1)
)

test_psd = FrequencySeries(
    np.ones(test_f_series.size) << 1/u.Hz,
    frequencies=np.arange(1, step=0.1)
)

In [16]:
# inner_product(test_f_series, test_t_series, test_psd)
# Gives error, but this is good! Because units are inconsistent in this definition

In [17]:
test_f_series = FrequencySeries(
    np.arange(1, step=0.1) << u.s,
    frequencies=np.arange(1, step=0.1)
)

test_t_series = TimeSeries(
    np.arange(1, step=0.1)
)

test_psd = FrequencySeries(
    np.ones(test_f_series.size) << 1/u.Hz,
    frequencies=np.arange(1, step=0.1)
)

In [18]:
inner_product(test_f_series, test_t_series, test_psd)

2024-01-31  15:20:12  INFO (inner_product.py: 303): s, s, s, 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 410): s, s, s, 1 / Hz


-0.08877475679537136

In [19]:
inner_product(test_f_series, test_t_series, test_psd, optimize_time_and_phase=True)

2024-01-31  15:20:12  INFO (inner_product.py: 303): s, s, s, 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 410): s, s, s, 1 / Hz
2024-01-31  15:20:12  INFO (inner_product.py: 425): s, s, s, 1 / Hz


(<TimeSeries([-0.01559455+0.04486263j,  0.07942271+0.02015154j,
              -0.01760464+0.04247489j,  0.04142973+0.02165699j,
              -0.02646153+0.04211833j,  0.02018382+0.02064279j,
              -0.04269432+0.04563933j, -0.01551484+0.00716573j,
              -0.05346179+0.00423435j,  0.01180083-0.00869187j,
              -0.0671692 +0.01358412j,  0.02434543-0.02270845j,
              -0.16083033+0.05000233j, -0.02024122-0.3034182j ,
               0.24238992+0.0222855j ]
             unit=Unit(dimensionless),
             t0=<Quantity -7.46666667 1 / Hz>,
             dt=<Quantity 1.06666667 1 / Hz>,
             name=None,
             channel=None)>,
 0.30409260406479216,
 <Quantity 7.46666667 1 / Hz>)

Note: this is just a test if both versions run. The results being unequal is not bad, after all test_f_series and test_t_series are not the same one and thus there will be shift so that their overlap is more than for no relative shift.